In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os

In [2]:
# Path to your dataset directory
data_dir = r"C:\Users\Sanika_almale\Desktop\12 Animal\raw-img"  # Replace with your local dataset path

In [3]:
# Load the MobileNetV2 model, excluding the top layer (output layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [4]:
# Freeze the base model to prevent updating the pre-trained weights during initial training
base_model.trainable = False

In [5]:
# Add new classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add global average pooling layer
x = Dense(128, activation='relu')(x)  # Add a fully-connected layer with 128 units
output_layer = Dense(11, activation='softmax')(x)  # Adjust output layer for number of classes (change 10 if different)

In [6]:
# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

In [7]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Data augmentation and normalization for training
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Rescale the pixel values to [0, 1]
    validation_split=0.2,  # Reserve 20% of the data for validation
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

In [9]:
# Prepare train and validation generators
train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 20995 images belonging to 11 classes.


In [10]:
validation_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 5244 images belonging to 11 classes.


In [11]:
# Set up a model checkpoint to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_animal_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

In [ ]:
# Train the model for a few epochs
history = model.fit(train_gen, validation_data=validation_gen, epochs=5, callbacks=[checkpoint])

C:\Users\Sanika_almale\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
657/657 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8729 - loss: 0.4241

C:\Users\Sanika_almale\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_loss improved from inf to 0.17748, saving model to best_animal_model.keras
657/657 ━━━━━━━━━━━━━━━━━━━━ 1522s 2s/step - accuracy: 0.8730 - loss: 0.4239 - val_accuracy: 0.9424 - val_loss: 0.1775
Epoch 2/5
136/657 ━━━━━━━━━━━━━━━━━━━━ 16:57 2s/step - accuracy: 0.9373 - loss: 0.1849

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_gen)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
# Visualize some predictions
x_val, y_val = next(validation_gen)  # Get a batch of validation data
predictions = model.predict(x_val)
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
# Get class labels
class_labels = list(train_gen.class_indices.keys())

In [ ]:
# Plot actual vs predicted labels for the first 5 images
n = 5
plt.figure(figsize=(10, 20))
for i in range(n):
    plt.subplot(n, 1, i + 1)
    plt.imshow(x_val[i])
    actual_label = class_labels[np.argmax(y_val[i])]
    predicted_label = class_labels[predicted_labels[i]]
    plt.title(f"Actual: {actual_label}, Predicted: {predicted_label}")
    plt.axis('off')
plt.tight_layout()
plt.show()